In [ ]:
from google import genai
from google.genai import types
from google.genai.errors import ClientError
from io import BytesIO
from dotenv import load_dotenv
import os
import PIL.Image
import time

load_dotenv()
    
MAX_RETRIES = 5
attempts = 0

image = PIL.Image.open('sample_images\sample_4.jpg')

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

text_input = ('The given is an image of a piece of clothing. I want you to generate a ultrarealistic image of the clothing on a person.',
              'You must strictly make sure that the clothing is represented exactly how is given in the reference image.',
              'You must show the person completely from head to toe and if any necessary piece of clothing is missing (such as a bottom from an image of a top), you may add as appropriate.',
              'The generated image is to be used to display the item on an ecommerce website so keep that in mind when generating it.')

while attempts < MAX_RETRIES:
  try:
    response = client.models.generate_content(
        model="gemini-2.0-flash-preview-image-generation",
        contents=[text_input, image],
        config=types.GenerateContentConfig(
            response_modalities=['TEXT', 'IMAGE'],
            temperature=0.5
        )
    )

    output = {"text": None, "image": None}
    for part in response.candidates[0].content.parts:
        if part.text is not None:
            output["text"] = part.text
        elif part.inline_data is not None:
            image = BytesIO((part.inline_data.data))
            output["image"] = image

    display(output)
    break
  
  except ClientError as e:
    if str(e).startswith("429 RESOURCE_EXHAUSTED"):
      print("You have exceeded todays quota.")
      break
  
  except Exception as e:
    attempts += 1
    time.sleep(30)
    
else:
  print(f"Too many errors of type {type(e)} with error message {str(e)}")

You have exceeded todays quota.
